# Setting up

## Modules

In [1]:
%env

{'CLICOLOR': '1',
 'GIT_PAGER': 'cat',
 'HOME': '/home/mark_zwart76',
 'JPY_PARENT_PID': '2079',
 'LANG': 'en_US.UTF-8',
 'LESSCLOSE': '/usr/bin/lesspipe %s %s',
 'LESSOPEN': '| /usr/bin/lesspipe %s',
 'LOGNAME': 'mark_zwart76',
 'LS_COLORS': 'rs=0:di=01;34:ln=01;36:mh=00:pi=40;33:so=01;35:do=01;35:bd=40;33;01:cd=40;33;01:or=40;31;01:mi=00:su=37;41:sg=30;43:ca=30;41:tw=30;42:ow=34;42:st=37;44:ex=01;32:*.tar=01;31:*.tgz=01;31:*.arc=01;31:*.arj=01;31:*.taz=01;31:*.lha=01;31:*.lz4=01;31:*.lzh=01;31:*.lzma=01;31:*.tlz=01;31:*.txz=01;31:*.tzo=01;31:*.t7z=01;31:*.zip=01;31:*.z=01;31:*.Z=01;31:*.dz=01;31:*.gz=01;31:*.lrz=01;31:*.lz=01;31:*.lzo=01;31:*.xz=01;31:*.bz2=01;31:*.bz=01;31:*.tbz=01;31:*.tbz2=01;31:*.tz=01;31:*.deb=01;31:*.rpm=01;31:*.jar=01;31:*.war=01;31:*.ear=01;31:*.sar=01;31:*.rar=01;31:*.alz=01;31:*.ace=01;31:*.zoo=01;31:*.cpio=01;31:*.7z=01;31:*.rz=01;31:*.cab=01;31:*.jpg=01;35:*.jpeg=01;35:*.gif=01;35:*.bmp=01;35:*.pbm=01;35:*.pgm=01;35:*.ppm=01;35:*.tga=01;35:*.xbm=01;35:*.x

In [1]:
import sys
sys.path.append("../modules")

In [2]:
from cleaning_merging import *

In [3]:
from datetime import datetime
from IPython.display import HTML

## Data gathering variables

In [4]:
date_dataset = datetime.strptime("2018-01-01", '%Y-%m-%d')

### For a loop

In [5]:
dates = ["2018-01-01", "2017-01-01", "2016-01-01", "2015-01-01", "2014-01-01", "2013-01-01"]
list_date_dataset = [datetime.strptime(date, '%Y-%m-%d') for date in dates]

## Project constants

In [6]:
project = 'graydon-moving-indicator'
bucket_name = 'graydon-data'
dir_input_data = '01_input'
dir_output_data = '02_cleaned'

# Gathering data

Instantiate clean and merge object

In [7]:
clean_merge = Cleaner_Merger(project, bucket_name, dir_input_data, dir_output_data)

In [10]:
df = clean_merge.get_df_from_bucket(dir_input_data + "/additional_data/location_start_date.CSV",
                           sep=',', na_values=['', '1198-06-12', 'NA'],
                           parse_dates=['date_relocation_last', 'date_relocation_penultimate'])

In [ ]:
df.dtypes

## Features

In [ ]:
df_features = clean_merge.get_features(date_dataset)

Example of the data:

In [ ]:
df_features.head()

Number of company, branch, month combinations

In [ ]:
df_features.shape[0]

## Target

In [ ]:
df_target = clean_merge.get_targets(date_dataset, columns_targets)

Example of the data:

In [ ]:
df_target.head()

The number of companies with target:

In [ ]:
df_target.shape[0]

## Combining features and target

In [ ]:
df_monthly = df_features.merge(df_target,
                               on=['id_company', 'id_branch'],
                               how='left')

Example of the data:

In [ ]:
df_monthly.head()

Number of monthly company branch data records:

In [ ]:
df_monthly.shape[0]

In [ ]:
df_monthly['has_relocated']

## -----------

In [ ]:
date_month = date_dataset

df_months_combined = pd.DataFrame()  # The data frame which will contain all independent variables
    
# Get all months in range
df_date_months = pd.DataFrame(pd.date_range(date_month, periods=12, freq="M").tolist(),
                              columns=['date_month'])
df_date_months['date_month'] = df_date_months['date_month'].values.astype('datetime64[M]') # First day of month

# Get the file names of all required month files
#month_files = get_month_filenames(df_date_months, bucket, dir_data)

In [ ]:
month_files = [] # List of month files

df_date_months['year'] = df_date_months.date_month.dt.year
list_years = df_date_months['year'].unique()
# If there are multiple years, iterate through years  
for year in list_years:
    # Get the year's data file names
    dir_data_year = dir_data + '/' + str(year)
    list_blob = list(bucket.list_blobs(prefix=dir_data_year))
    # finding out which month files should be processed by looking which contain the first month date (YYYY-mm-01)
    df_year_months = df_date_months[df_date_months['year'] == year]['date_month']
    for blob in list_blob:
        for month in df_year_months:
            if (month.strftime("%Y-%m-%d") in blob.name) & ('CSV' in blob.name):
                month_files.append(blob.name)

In [ ]:
month_files

In [ ]:
# Cleaning, transforming and combining month files                
for month_file in month_files:
    with fs.open('graydon-data/' + month_file) as f:
        df_month = pd.read_csv(f, sep=';', usecols= columns_features, index_col=False)   
        df_month = df_month[(df_month['is_sole_proprietor'] == 0)] # & (one_month_df['is_discontinued'] == 0) 
        df_month.columns = (df_month.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', ''))
        df_months_combined = df_months_combined.append(df_month)
        print('The number of rows so far by adding ', month_file, ":", df_months_combined.shape[0])

df_months_combined['date_dataset'] = date_month

# Aggregating data to year
df_months_combined = df_months_combined.groupby(['date_dataset', 
                                                  'id_company', 
                                                  'id_branch']).agg({'has_relocated': 'max', 
                                                                     'date_month': 'max'})
df_months_combined = df_months_combined.rename(index=str, columns={"date_month": "date_month_last"})
df_months_combined = df_months_combined.reset_index()
df_months_combined['date_dataset'] = pd.to_datetime(df_months_combined['date_dataset'])
df_months_combined['id_company'] = df_months_combined['id_company'].astype(int)
df_months_combined['id_branch'] = df_months_combined['id_branch'].astype(int)

return(df_months_combined)